In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [6]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurement', 'station']

In [7]:
meas_columns = inspector.get_columns('measurement')
for column in meas_columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [8]:
station_columns = inspector.get_columns('station')
for column in station_columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [9]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
# reflect the tables


['measurement', 'station']

In [10]:
# View all of the classes that automap found


In [11]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [12]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [13]:
# Find the most recent date in the data set.
first_row = session.query(Measurement).first()
max_date = session.query(func.max(Measurement.date)).scalar()
date_split = max_date.split("-")

In [62]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

min_date = dt.date(int(date_split[0]), int(date_split[1]), int(date_split[2])) - dt.timedelta(days=365)






# Perform a query to retrieve the data and precipitation scores

results = session.query(Measurement.date, Measurement.prcp).\
                 filter(Measurement.date > min_date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date

df = pd.DataFrame(results, columns=['date', 'precipitation'])
clean_df = df.dropna(how="any")
clean_df.set_index('date', inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data

sorted_df = clean_df.sort_values(by="date")
sorted_df



dates = sorted_df.index.to_list()

unq_dates = set(dates)

unq_date_list = []
for item in unq_dates:
    unq_date_list.append(item)
    





In [63]:
unq_date_list.sort()
unq_date_list
precip = sorted_df["precipitation"].to_list()


In [64]:
labels = unq_date_list[::30]
labels.append(dates[-1])
labels

['2016-08-24',
 '2016-09-23',
 '2016-10-23',
 '2016-11-22',
 '2016-12-22',
 '2017-01-21',
 '2017-02-20',
 '2017-03-22',
 '2017-04-21',
 '2017-05-21',
 '2017-06-20',
 '2017-07-20',
 '2017-08-19',
 '2017-08-23']

In [ ]:
fig, prec = plt.subplots()
    
prec.bar(dates, precip, color="lightsalmon")

# force the location of the xticks -- monthly ticks are too many to read on the x axis.
# label with the year, instead
x_formatter = FixedFormatter(labels)
x_locator = FixedLocator([0, 28, 56, 84, 112, 140, 168, 196, 224, 252, 280, 308, 336, 365])

prec.xaxis.set_major_formatter(x_formatter)
prec.xaxis.set_major_locator(x_locator)

y_labels = [0, 1, 2, 3, 4, 5, 6, 7]
y_formatter = FixedFormatter(y_labels)
y_locator = FixedLocator(y_labels)

prec.yaxis.set_major_formatter(y_formatter)
prec.yaxis.set_major_locator(y_locator)

plt.ylabel("Inches", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.xticks(fontsize=10, rotation=90)
plt.yticks(fontsize=10)


plt.tight_layout()
plt.savefig("output/precipitation_bar.png")

plt.show()

In [15]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

,precipitation
count,2015.000000
mean,0.176462
std,0.460288
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()